## [Rohlik | Abd_Base [ CV : 20.0640, LB : 22.94348 ]](https://www.kaggle.com/code/abdmental01/rohlik-abd-base-cv-20-0640-lb-22-94348)

In [11]:
import numpy as np
import polars as pl
import pandas as pd
from sklearn.base import clone
import optuna
import os
from tqdm import tqdm
import category_encoders as ce
from IPython.display import clear_output

import warnings
warnings.filterwarnings('ignore')
pd.options.display.max_columns = None

import lightgbm as lgb
from lightgbm import early_stopping  
from catboost import CatBoostRegressor, CatBoostClassifier, Pool
from sklearn.model_selection import *
from sklearn.metrics import *

### 1. data 병합

In [12]:
%%time

import pandas as pd

train = pd.read_csv('../../data/sales_train.csv')
test = pd.read_csv('../../data/sales_test.csv')
sample = pd.read_csv('../../data/solution.csv')
inv = pd.read_csv('../../data/inventory.csv')
cle = pd.read_csv('../../data/calendar.csv')
test_weights = pd.read_csv('../../data/test_weights.csv')


train = train.merge(cle,on=['warehouse','date'],how='left') 
train = train.merge(inv,on=['warehouse','unique_id'],how='left')

test = test.merge(cle,on=['warehouse','date'],how='left')
test = test.merge(inv,on=['warehouse','unique_id'],how='left')

train = train.dropna(subset=['sales']) # Null in Target

train.drop('availability', axis=1, inplace=True) # Not Available in Test
train.drop('name', axis=1, inplace=True) # Creating Problem in Training
test.drop('name', axis=1, inplace=True) 

def date(df):
    
    df['date'] = pd.to_datetime(df['date'])
    df['year'] = df['date'].dt.year
    df['day'] = df['date'].dt.day
    df['month'] = df['date'].dt.month
    df['month_name'] = df['date'].dt.month_name()
    df['day_of_week'] = df['date'].dt.day_name()
    df['week'] = df['date'].dt.isocalendar().week
    df['year_sin'] = np.sin(2 * np.pi * df['year'])
    df['year_cos'] = np.cos(2 * np.pi * df['year'])
    df['month_sin'] = np.sin(2 * np.pi * df['month'] / 12) 
    df['month_cos'] = np.cos(2 * np.pi * df['month'] / 12)
    df['day_sin'] = np.sin(2 * np.pi * df['day'] / 31)  
    df['day_cos'] = np.cos(2 * np.pi * df['day'] / 31)
    df['group'] = (df['year'] - 2020) * 48 + df['month'] * 4 + df['day'] // 7
    
    df.drop('date', axis=1, inplace=True)
    
    cols = ['warehouse', 'month_name', 'day_of_week','holiday_name','L1_category_name_en',
             'L2_category_name_en','L3_category_name_en','L4_category_name_en']
    df['holiday_name'] = df['holiday_name'].fillna('None')
    for c in cols:
        df[c] = df[c].astype('category')

    return df


train = date(train)
test = date(test)

In [13]:
train.head()

CPU times: total: 0 ns
Wall time: 0 ns


,unique_id,warehouse,total_orders,sales,sell_price_main,type_0_discount,type_1_discount,type_2_discount,type_3_discount,type_4_discount,type_5_discount,type_6_discount,holiday_name,holiday,shops_closed,winter_school_holidays,school_holidays,product_unique_id,L1_category_name_en,L2_category_name_en,L3_category_name_en,L4_category_name_en,year,day,month,month_name,day_of_week,week,year_sin,year_cos,month_sin,month_cos,day_sin,day_cos,group
0,4845,Budapest_1,6436.0,16.34,646.26,0.00000,0.0,0.0,0.0,0.15312,0.0,0.0,None,0,0,0,0,2375,Bakery,Bakery_L2_18,Bakery_L3_83,Bakery_L4_1,2024,10,3,March,Sunday,10,1.585375e-14,1.0,1.000000e+00,6.123234e-17,0.897805,-0.440394,205
1,4845,Budapest_1,4663.0,12.63,455.96,0.00000,0.0,0.0,0.0,0.15025,0.0,0.0,None,0,0,0,0,2375,Bakery,Bakery_L2_18,Bakery_L3_83,Bakery_L4_1,2021,25,5,May,Tuesday,21,-1.468363e-13,1.0,5.000000e-01,-8.660254e-01,-0.937752,0.347305,71
2,4845,Budapest_1,6507.0,34.55,455.96,0.00000,0.0,0.0,0.0,0.15025,0.0,0.0,None,0,0,0,0,2375,Bakery,Bakery_L2_18,Bakery_L3_83,Bakery_L4_1,2021,20,12,December,Monday,51,-1.468363e-13,1.0,-2.449294e-16,1.000000e+00,-0.790776,-0.612106,98
3,4845,Budapest_1,5463.0,34.52,646.26,0.20024,0.0,0.0,0.0,0.15312,0.0,0.0,None,0,0,0,0,2375,Bakery,Bakery_L2_18,Bakery_L3_83,Bakery_L4_1,2023,29,4,April,Saturday,17,-6.447061e-13,1.0,8.660254e-01,-5.000000e-01,-0.394356,0.918958,164
4,4845,Budapest_1,5997.0,35.92,486.41,0.00000,0.0,0.0,0.0,0.15649,0.0,0.0,None,0,0,0,0,2375,Bakery,Bakery_L2_18,Bakery_L3_83,Bakery_L4_1,2022,1,4,April,Friday,13,-1.305266e-12,1.0,8.660254e-01,-5.000000e-01,0.201299,0.979530,112


[ABDBASE || LGBM](https://github.com/muhammadabdullah0303/AbdML) <= 소스코드

In [15]:
from AbdML.main import AbdBase

In [16]:
%%time

weights = test_weights['weight']

SEED = 42

print("*** AbdBase V_1.1 ***\n")
base = AbdBase(train_data=train, test_data=test, target_column='sales',gpu=True,
                 problem_type="regression", metric="wmae", seed=SEED,
                 n_splits=5,early_stop=True,num_classes=0,prob=False,
                 fold_type='KF',weights=weights,tf_vec=False)

*** AbdBase V_1.1 ***

*** AbdBase ['V_1.3'] ***

 *** Available Settings *** 

Available Models: LGBM, CAT, XGB, Voting, TABNET, Ridge, LR
Available Metrics: roc_auc, accuracy, f1, precision, recall, rmse, wmae, rmsle, mae, r2, mse, mape, custom
Available Problem Types: classification, regression
Available Fold Types: SKF, KF, GKF, GSKF, RKF

 *** Configuration *** 

Problem Type Selected: REGRESSION
Metric Selected: WMAE
Fold Type Selected: KF
Calculate Train Probabilities: False
Calculate Test Probabilities: False
Early Stopping: True
GPU: True
Eval_Metric Selected is: None
CPU times: total: 422 ms
Wall time: 453 ms


In [17]:
%%time

Params = {'n_estimators': 1811, 'learning_rate': 0.28570714885887566, 'max_depth': 14, 'num_leaves': 140,'n_jobs':-1,
          'min_child_samples': 19, 'subsample': 0.7467983561008608, 'colsample_bytree': 0.7174250836504598,
          'lambda_l1': 5.3994844097874335, 'lambda_l2': 1.5930522616241019, 'min_gain_to_split': 0.21242177333881365}

meanOFF, meanTest = base.Train_ML(Params,'LGBM',e_stop=50)

The EarlyStopping is 50


Training Folds:   0%|          | 0/5 [00:00<?, ?it/s]

In [ ]:
%%time

sample['sales_hat'] = meanTest

sample.to_csv('Submission_.csv', index = False)
sample.head()